# MicroKWS Flow [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tum-ei-eda/micro-kws/blob/workshop/MicroKWS.ipynb)

In [ ]:
# !pip install -U tensorboard_plugin_profile

Complete flow for keyword spotting on microcontrollers. From data collection to data preparation to training and deployment.

## 0. Prerequisites

**MicroKWS Repository**
Please clone the `micro-kws` repository into the Colab session by executing the following cell:

In [ ]:
!test -d /content/micro-kws || git clone https://github.com/tum-ei-eda/micro-kws --recursive -b workshop /content/micro-kws

**Jupyter Notebook**
If running this notebook locally (e.g. not in Google Colab) some initial steps have to be done after cloning the `micro-kws` repository:
- Create & enter virtual Python environment
- Install Python packages (see `requirments.txt`) into environment
- Start jupyter notebook server

Allow reloading modules without kernel restarts

In [ ]:
%load_ext autoreload
%autoreload 2

**Google Colab**

To reduce training time, make sure to select a **GPU** runtime in the menu `Runtime -> Change runtime type`.

Execute one cell at the time from top to bottom.

Indivitial sections can be hided if not required.

Cells starting with a `!` or `%` are shell commands. The others contain Python code.

Next change into the cloned directory

In [ ]:
%cd /content/micro-kws/

Google Colab privides a *Table of contents* feature in the left sidebar which should be very useful for navigating between the sections of the Notebook. Indvidual seconds can be collapsed for getting a better overview. The sidebar can also be used to access the *Files* explorer. Feels free to look into any of the files used in the following steps. Here you can also download the generated files at the end of each section.

Some python dependencies should be downloaded in advance:

In [ ]:
!pip install netron

In [ ]:
import sys
sys.path.insert(0, '/content/micro-kws/train')

## 1. Data Generation & Preprocessing

In [ ]:
%cd /content/micro-kws/train

The download procedure and pre-processing of the dataset is part of the following section and therefore not covered seperately here.



### 1.1 Speech commands dataset

Import some Python helper functions first:

In [ ]:
from utils import playAudio, showWaveform

The used dataset is explained in a paper which can be found [here](https://arxiv.org/abs/1804.03209).

It can be summarized as follows:

- Audio dataset of spoken words collected in a community project in 2017 designed to help train and evaluate keyword spotting systems.
- Size: 8.17 GiB, ∼ 100k WAV files
- 34 Keywords, ~3,000 samples each
- Special labels: unknown, silence, (background noise)


Let's download the dataset manually using the following command:

In [ ]:
!wget http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz

Next, we have to extract it:

In [ ]:
!mkdir -p speech_dataset
!tar xf speech_commands_v0.02.tar.gz -C speech_dataset
%env SPEECH_COMMANDS_DIR=speech_dataset

The supported keywords of the dataset can be listed as follows:

In [ ]:
!ls speech_dataset/

Here we can see some of the samples for the 'yes' keyword:

In [ ]:
!ls speech_dataset/yes/ | head

We can also listen to one of the samples:

In [ ]:
SAMPLE = "speech_dataset/yes/51995cea_nohash_0.wav"
playAudio(SAMPLE)

A rendered waveform of the raw samples looks like this:

In [ ]:
showWaveform(SAMPLE)

### 1.2 Recording custom samples

To generate own speech samples for extend the dataset, we provide a tool called *TUM Speech Recording* which is available [here](https://github.com/tum-ei-eda/tum-speech-recording/tree/main).



**Usage (Windows/MacOS/Ubuntu):**

Open a shell with Python v3.7 or higher installed (Virtual/Conda Environment recommended) and first enter the `record` subdirectory of the `micro-kws` repository and install the required packages:

```shell
pip install -r requirements.txt
```

Now, run `python app.py` and the speech recorder should be available under `127.0.0.1:8080` in your web browser.

![screen1.PNG](https://github.com/tum-ei-eda/micro-kws/raw/workshop/resources/recorder/screen1.PNG)

## 2. Model Design & Training

This section should give an introduction on the required procedure to design, train and quantize a small machine learning model using the Tensorflow Lite and the Keras API. The application example is a keyword-spotting (KWS) task which should ideally be suitable to run on an energy efficent device e.g. a small microcontroller platform.

### 2.1 Import Python dependencies**

Python builtin dependencies

In [ ]:
import os
import sys
import tempfile

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"  # Reduce verbosity
import argparse
from pathlib import Path

Third party dependencies

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

Helper scripts python files in (see `micro-kws/train/`)

In [ ]:
import data
import models
from test import get_accuracy, get_confusion_matrix
from test_tflite import tflite_test
from utils import visualizeFeature, showNetron

Enable Tensorboard support

In [ ]:
%load_ext tensorboard

### 2.2 Define training parameters

In this section the hyperparameters for the model and training procedure are defined. Feel free to read through the code line by line as the options should be all documented well.

In [ ]:
FLAGS = argparse.Namespace()

# Overwrite the model name provided by keras with a custom one
FLAGS.model_name = "micro_kws_m"

# Location of speech training data archive on the web.
FLAGS.data_url = "http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz"

# Where to download the speech training data to.
try:
    login = os.getlogin()
except:
    login = "unknown"
FLAGS.data_dir = os.getenv(
    "SPEECH_COMMANDS_DIR",
    default=os.path.join(tempfile.gettempdir(), login, "speech_dataset"),
)

# Words to use (others will be added to an unknown label)
FLAGS.wanted_words = "yes,no,up,down,left,right,on,off"

# Total number of classifiations labels (wanted_words + silence + unknown)
FLAGS.num_classes = len(FLAGS.wanted_words.split(",")) + 2

# How loud the background noise should be, between 0 and 1.
FLAGS.background_volume = 0.1

# How many of the training samples have background noise mixed in.
FLAGS.background_frequency = 0.8

# How much of the training data should be silence.
FLAGS.silence_percentage = 100.0 / FLAGS.num_classes

# How much of the training data should be unknown words
FLAGS.unknown_percentage = 100.0 / FLAGS.num_classes

# Range to randomly shift the training audio by in time.
FLAGS.time_shift_ms = 100.0

# What percentage of wavs to use as a test set.
FLAGS.testing_percentage = 10

# What percentage of wavs to use as a validation set.
FLAGS.validation_percentage = 10

# Expected sample rate of the wavs
FLAGS.sample_rate = 16000

# Expected duration in milliseconds of the wavs
FLAGS.clip_duration_ms = 1000

# How long each spectrogram timeslice is
FLAGS.window_size_ms = 30.0

# How long each spectrogram timeslice is
FLAGS.window_stride_ms = 20.0

# How many bins to use for the MFCC fingerprint
FLAGS.dct_coefficient_count = 40

# How many training loops to run
# FLAGS.how_many_training_steps = "12000,3000"
FLAGS.how_many_training_steps = "2000,500"  # Reduced to make training faster

# How often to evaluate the training results.
FLAGS.eval_step_interval = 400

# How large a learning rate to use when training.
FLAGS.learning_rate = "0.001,0.0001"

# How many items to train with at once
FLAGS.batch_size = 100

# Where to save summary logs for TensorBoard.
# FLAGS.summaries_dir = '/tmp/retrain_logs'

# Directory to write event logs and checkpoint.
FLAGS.train_dir = "training"

# Directory to write tensorboard logs to.
FLAGS.logs_dir = "logs"

# Directory to write converted models to.
FLAGS.models_dir = "models"

# Use tensorboard feature to visualize training process
FLAGS.tensorboard = True

# Profile the training procedure (WIP)
FLAGS.profile = True

Get the model settings as they are required for the preprocessing, training and quantization.

In [ ]:
model_settings = models.prepare_model_settings(
    len(data.prepare_words_list(FLAGS.wanted_words.split(","))),
    FLAGS.sample_rate,
    FLAGS.clip_duration_ms,
    FLAGS.window_size_ms,
    FLAGS.window_stride_ms,
    FLAGS.dct_coefficient_count,
)

### 2.3 Prepare dataset

While keyword-spotting is quite simple tasks, the preprocessing to generate input features for training is non-trivial. Hence, the implementation of the `Audioprocessor()` class is omited here. If interested, check out the `data.py` script for more information.

One aspect, which is very important here is the `micro=True` option as it ensures that the same preprocessing (conversion of input WAV files to an Image) is applied to the input dataset as used in the mcirocontroller target software.

*Warning:* The execution of the following cell might take more than 10 minutes, depending on the available download bandwidth.

In [ ]:
 audio_processor = data.AudioProcessor(
    data_url=FLAGS.data_url,
    data_dir=FLAGS.data_dir,
    silence_percentage=FLAGS.silence_percentage,
    unknown_percentage=FLAGS.unknown_percentage,
    wanted_words=FLAGS.wanted_words.split(","),
    validation_percentage=FLAGS.validation_percentage,
    testing_percentage=FLAGS.testing_percentage,
    model_settings=model_settings,
    micro=True,
)

Execute the following cell a few times to inpect the generated features for some keywords/labels.

In [ ]:
feature = audio_processor.get_data(audio_processor.Modes.VALIDATION).shuffle(100).take(1).get_single_element()
visualizeFeature(feature, FLAGS)

### 2.4 Create Keras Model

Define a model architecture using the Keras API

In [ ]:
def create_model(model_settings):
    """Builds a model with a single depthwise-convolution layer followed by a single fully-connected layer.
    Args:
        model_settings: Dict of different settings for model training.
    Returns:
        tf.keras Model of the 'micro_kws_xs' architecture.
    """

    # Get relevant model setting.
    input_frequency_size = model_settings["dct_coefficient_count"]
    input_time_size = model_settings["spectrogram_length"]

    inputs = tf.keras.Input(shape=(model_settings["fingerprint_size"]), name="input")

    # Reshape the flattened input.
    x = tf.reshape(inputs, shape=(-1, input_time_size, input_frequency_size, 1))

    # First convolution.
    x = tf.keras.layers.DepthwiseConv2D(
        depth_multiplier=16,
        kernel_size=(20, 8),
        strides=(1, 1),
        padding="SAME",
        activation="relu",
    )(x)

    x = tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2),
    )(x)

    # Second convolution.
    x = tf.keras.layers.Conv2D(
        # filters=8,
        filters=16,
        kernel_size=(10, 4),
        strides=(1, 1),
        padding="SAME",
        activation="relu",
    )(x)

    # Flatten for fully connected layers.
    x = tf.keras.layers.Flatten()(x)

    # Output fully connected.
    output = tf.keras.layers.Dense(units=model_settings["label_count"], activation="softmax")(x)

    return tf.keras.Model(inputs, output, name=FLAGS.model_name)

Generate keras model. The `model.summary()` utility provides a way to inspect the layers of Keras model with its shapes and parameters.

In [ ]:
model = create_model(model_settings)
model.summary()

### 2.5 Run Training

Define training procedure using the previously defined parameters.

In addition to the training hyperparamerters (training steps, learning rate,...) an optimizer (`Adam`), a loss function (`SparseCategoricalCrossentropy`) as well as a metric is selected for the training and passed to the `model.compile()` method.

The actual training happens when `model.fit()` is called. The training progress should be visible on the screen. While multiple epochs ("one pass over the entire dataset") are required for the training, the validation accuracy is evaluated every 200 steps and the weights are written to a directory automatically.

At the end of the training procedure the final test accuracy of the trained model is printed to the screen.

In [ ]:
def train(model, audio_processor):
    # We decay learning rate in a constant piecewise way to help learning.
    training_steps_list = list(map(int, FLAGS.how_many_training_steps.split(",")))
    learning_rates_list = list(map(float, FLAGS.learning_rate.split(",")))
    lr_boundary_list = training_steps_list[:-1]  # Only need the values at which to change lr.
    lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
        boundaries=lr_boundary_list, values=learning_rates_list
    )

    # Specify the optimizer configurations.
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    # Compile the model.
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )

    # Prepare/split the dataset.
    train_data = audio_processor.get_data(
        audio_processor.Modes.TRAINING,
        FLAGS.background_frequency,
        FLAGS.background_volume,
        int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000),
    )
    train_data = train_data.repeat().batch(FLAGS.batch_size).prefetch(tf.data.AUTOTUNE)
    val_data = audio_processor.get_data(audio_processor.Modes.VALIDATION)
    val_data = val_data.batch(FLAGS.batch_size).prefetch(tf.data.AUTOTUNE)

    # We train for a max number of iterations so need to calculate how many 'epochs' this will be.
    training_steps_max = np.sum(training_steps_list)
    training_epoch_max = int(np.ceil(training_steps_max / FLAGS.eval_step_interval))

    # Callbacks.
    callbacks = []
    train_dir = Path(FLAGS.train_dir) / FLAGS.model_name / "best"
    train_dir.mkdir(parents=True, exist_ok=True)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=(train_dir / (FLAGS.model_name + "_{val_accuracy:.3f}_ckpt")),
        save_weights_only=True,
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
    )
    callbacks.append(model_checkpoint_callback)
    # TODO: profiler
    if FLAGS.tensorboard:
        log_dir = Path(FLAGS.logs_dir) / FLAGS.model_name
        log_dir.mkdir(parents=True, exist_ok=True)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1, profile_batch='500,520' if FLAGS.profile else 0)
        callbacks.append(tensorboard_callback)

    # Train the model.
    model.fit(
        x=train_data,
        steps_per_epoch=FLAGS.eval_step_interval,
        epochs=training_epoch_max,
        validation_data=val_data,
        callbacks=callbacks,
    )

    # Test and save the model.
    test_data = audio_processor.get_data(audio_processor.Modes.TESTING)
    test_data = test_data.batch(FLAGS.batch_size)

    # Evaluate the model performance.
    test_loss, test_acc = model.evaluate(x=test_data)
    print(f"Final test accuracy: {test_acc*100:.2f}%")

In the following cell the training progress will be shown using a few useful graphs.

In [ ]:
if FLAGS.tensorboard:
    log_dir = Path(FLAGS.logs_dir) / FLAGS.model_name
    log_dir.mkdir(parents=True, exist_ok=True)
    %tensorboard --logdir {log_dir} --host=0.0.0.0

Invoke training procedure (**Warning:** This will take a very long time! The number of epochs hase been reduced too keep the time down, however this will have an impact in the resulting accuracy.)

In [ ]:
 train(model, audio_processor)

Determine latest checkpoint

In [ ]:
latest = tf.train.latest_checkpoint(Path(FLAGS.train_dir) / FLAGS.model_name / "best")
print(latest)

Pick a checkpoint

In [ ]:
 FLAGS.checkpoint = latest  # Feel free to choose a different one!

### 2.6 Test trained TensorFlow model

Define test procedure we can use to evaluate our models performance.

The used test routines are defined in `test.py`.

In [ ]:
def test(model, audio_processor, model_settings):
    """Calculate accuracy and confusion matrices on validation and test sets.

    Model is created and weights loaded from supplied command line arguments.
    """
    model.load_weights(FLAGS.checkpoint).expect_partial()

    # Get test data
    data = audio_processor.get_data(audio_processor.Modes.TESTING).batch(FLAGS.batch_size)

    # Invoke model
    predictions = model.predict(data)

    # Calculate indices
    expected_indices = np.concatenate([y for x, y in data])
    predicted_indices = tf.argmax(predictions, axis=1)

    print("Running testing on test set...")
    accuracy = get_accuracy(expected_indices, predicted_indices)
    confusion_matrix = get_confusion_matrix(expected_indices, predicted_indices, model_settings)

    # Print accuracy and confusion matrix
    print(
        f"test accuracy = {accuracy * 100:.2f}%"
        f"(N={audio_processor.set_size(audio_processor.Modes.TESTING)})"
    )
    print()
    print("confusion matrix:")
    print(confusion_matrix.numpy())

Run test procedure

In [ ]:
test(model, audio_processor, model_settings)

Confusion matrices are also printed as they provide infomation about how the individual classes have performed.

## 3. Quantization and Conversion to TFLite

### 3.1 Post-training quantization

Define conversion procedure using the `TFLiteConverter` which creates a `.tflite` file which holds the model graph and constant weights.

In [ ]:
NUM_REP_DATA_SAMPLES = 100  # Number of representative samples which will be used for the post-training quantization.


def convert(model, audio_processor, checkpoint, quantize, inference_type, tflite_path):
    """Load our trained floating point model and convert it.
    TFLite conversion or post training quantization is performed and the
    resulting model is saved as a TFLite file.
    We use samples from the validation set to do post training quantization.
    Args:
        model: The keras model.
        audio_processor: Audio processor class object.
        checkpoint: Path to training checkpoint to load.
        quantize: Whether to quantize the model or convert to fp32 TFLite model.
        inference_type: Input/output type of the quantized model.
        tflite_path: Output TFLite file save path.
    """
    model.load_weights(checkpoint).expect_partial()

    val_data = audio_processor.get_data(audio_processor.Modes.VALIDATION).batch(1)

    def _rep_dataset():
        """Generator function to produce representative dataset."""
        i = 0
        for mfcc, label in val_data:
            if i > NUM_REP_DATA_SAMPLES:
                break
            i += 1
            yield [mfcc]

    converter = tf.lite.TFLiteConverter.from_keras_model(model)


    if quantize:
        # Quantize model and save to disk.
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        if inference_type == "int8":
            converter.inference_input_type = tf.int8
            converter.inference_output_type = tf.int8

        # Int8 post training quantization needs representative dataset.
        converter.representative_dataset = _rep_dataset
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

    tflite_model = converter.convert()
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)
    print("{} model saved to {}.".format("Quantized" if quantize else "Converted", tflite_path))

Invoke the previously defined conversion routine

In [ ]:
keywords_str = FLAGS.wanted_words.replace(",", "")
tflite_path_quantized = Path(FLAGS.models_dir) / f"{FLAGS.model_name}_{keywords_str}_quantized.tflite"
tflite_path = Path(FLAGS.models_dir) / f"{FLAGS.model_name}_{keywords_str}.tflite"

# Load floating point model from checkpoint and convert it.
convert(model, audio_processor, FLAGS.checkpoint, False, "fp32", tflite_path)

# Quantize model from checkpoint and convert it.
convert(model, audio_processor, FLAGS.checkpoint, True, "int8", tflite_path_quantized)

Compare files sizes of the floating point and quantized model:

In [ ]:
!du -h ./models/micro_kws_m_yesnoupdownleftrightonoff.tflite ./models/micro_kws_m_yesnoupdownleftrightonoff_quantized.tflite

The model architecture graph can be visualized as follows:

In [ ]:
showNetron(tflite_path_quantized)

### 3.2 Test Converted TFLite Model

**Floating Point**

In [ ]:
_ = tflite_test(model_settings, audio_processor, str(tflite_path))

**Quantized**

In [ ]:
_ = tflite_test(model_settings, audio_processor, str(tflite_path_quantized))

## 4. Model optimization and Code Generation with TVM

This section demonstrated how to optimized our MicroKWS TFLite model using the Python API provided by the TVM ML Compiler suite.

The same steps can also be performed on the commmand line. See `tutorial_tvmc.md` for more details!

In [ ]:
%cd /content/micro-kws/tvm

### 4.1 Prerequisites

If using a local machine, you might want to install the following packages

In [ ]:
#!sudo apt-get install python3 python3-dev python3-venv python3-setuptools gcc libtinfo-dev zlib1g-dev build-essential cmake libedit-dev libxml2-dev

Install python dependencies

In [ ]:
# pip install -r requirements.txt

In [ ]:
 !pip install pyusb pyserial tflite

Install TVM itself.

*Warning:* This is a nightly release and thing might break at some point in time!

In [ ]:
!pip install apache-tvm==0.11.1

### Compiling and optimizing the MicroKWS model with TVM

In [ ]:
!mkdir -p gen

MircoKWS deployment using TVMC command-line interface

This document will explain the required steps to process a model using the TVM machine learning compiler framework in detail. To generate the inference code for the application example of real-time keyword-spotting, the following steps have to be performed.While the following steps should work on modern versions of Windows, MacOS and Ubuntu, the complete flow was only tested on Ubuntu.Before continuing: Make sure that your virtual environment from the TVM installation step is active (sourced).

The term compile in the context of TVMC describes the complete compilation pipeline internally used by TVM (e.g. Relay frontend, Partitioning, Lowering, Code Generation etc.).

For execution on embedded device

The following (quite complex) command should be used to generate the TVM kernel implementations used in a later step:

In [ ]:
!tvmc compile /content/micro-kws/train/models/micro_kws_m_yesnoupdownleftrightonoff_quantized.tflite --output gen/mlf.tar \
    --target c --runtime crt --executor-aot-interface-api c \
    --executor aot --executor-aot-unpacked-api 1 --desired-layout NCHW \
    --output-format mlf --pass-config tir.disable_vectorize=1 \
    --pass-config tir.usmp.enable=1 --pass-config tir.usmp.algorithm=hill_climb

The used command line options can be explained as follows:

- `data/micro_kws_student.tflite`: The (quantized) TFLite model to process
- `--output gen/mlf.tar`: The destination file--target c: Tell TVM that we want to generate C kernels and not LLVM
- `--runtime crt`: Use the standalone CRT as we want to use a minimal runtime environment (e.g. baremetal code)
- `--executor-aot-interface-api c`: Generate a straightforward interface to define input and output tensors.
- `--executor aot`: Generate top-level model code using the Ahead-of-Time compiler to get rid of any runtime/interpreter related-overheads (Alternative: graph runtime using graph.json and params.bin)
- `--executor-aot-unpacked-api 1`: Use the \"unpacked\" calling convention for more compact code and less stack usage compared to TVM's default approach.
- `--desired-layout {NCHW,NHWC}`: Set the preferred layout of weights/kernels in the model (optional)
- `--output-format mlf`: Return MLF archive with codegen results. (Explained later)
- `--pass-config tir.disable_vectorize=1`: Disable optimizations which are not available on embedded platforms
- `--pass-config tir.usmp.enable=1`: Use the USMP (Unified Static Memory Planner) to minimize memory usage using a global tensor arena.
- `--pass-config tir.usmp.algorithm=hill_climb`: Select the algorithm used by the USMP (Alternatives: `greedy_by_size`, `greedy_by_conflicts`)

Further information on the available options can be found using the --help flag.


#### Using autotuned operators to build more efficient kernels

A major advantage of TVM's code-generation approach, besides the possibility to apply complex optimization at various abstraction layers, is the degree of freedom in the choice of the compute and schedule used to represent a given operator. While hand-crafted kernels (see TFLite for Microcontrollers) have to be as generic as possible to support a wide variety of different datatypes, shapes etc. TVM can choose from a number of possible parameterizable implementations for a given operator.

The challenge is to find the best implementation alongside a combination of parameters which has the "best" performance on a specific target device. An AutoTuner is provided by TVM to automate this process by exploring the search space using a number of exploration and optimization algorithms. As the autotuning procedure is quite time-intensive and requires a complex hardware/software setup, we will not invoke the AutoTuner here. Instead, we have done the autotuing for you. We provide the tuning records (`micro_kws_student_tuning_log_nchw_best.txt`) for the `data/micro_kws_student.tflite` model (see `tvm/data/`).

Add `--tuning-records data/micro_kws_student_tuning_log_nchw_best.txt` to the tvmc compile definition to use the tuning logs when compiling the model:

In [ ]:
!tvmc compile /content/micro-kws/train/models/micro_kws_m_yesnoupdownleftrightonoff_quantized.tflite --output gen/mlf_tuned.tar \
    --target c --runtime crt --executor-aot-interface-api c \
    --executor aot --executor-aot-unpacked-api 1 --desired-layout NCHW \
    --output-format mlf --pass-config tir.disable_vectorize=1 \
    --pass-config tir.usmp.enable=1 --pass-config tir.usmp.algorithm=hill_climb \
    --tuning-records data/micro_kws_student_tuning_log_nchw_best.txt

---

## 5. Deployment

Install espidf

In [ ]:
!git clone --recursive --single-branch -b release/v4.4 https://github.com/espressif/esp-idf.git

In [ ]:
!./esp-idf/install.sh esp32c3

In [ ]:
%env IDF_PATH=/content/micro-kws/tvm/esp-idf/

### 5.1 Deployment via MicroTVM

In [ ]:
 !. $IDF_PATH/export.sh && tvmc micro create prj ./gen/mlf.tar template \
    --template-dir espidf/template_project \
    --project-option project_type=micro_kws idf_target=esp32c3 num_classes=10

In [ ]:
 !. $IDF_PATH/export.sh && tvmc micro build prj template --template-dir espidf/template_project --project-option idf_target=esp32c3

next step: flash & run (on actual computer)

## 6. Evaluation

### 6.1 Real-world performance

After flashing the MicroKWS model to the MCU we first get an idea of the detection accuracy. It might be lower than the accuracy of the trained model for several reasons:

- Missmatch between used microphones
- Dataset not large enough
- Too much noise in the signal
- ...

We still can tune the performance of our MicroKWS i.e. by adapting the settings of the posterior handler. (Window size, threshold,...)

#### 6.2 Optional: MicroKWS Debugger GUI

We provide a *MicroKWS Debugger GUI* for visualizing the processes features and raw model output in real-time. See `micro-kws/debug/`

**Usage:**

Enter the `debug` subdirectory in the `micro-kws` repository in a shell with Python3.8+ installed (Virtual/Conda environment) and first install all the dependencies:

```shell
pip install -r requirements.txt
```

Start the debugger with the following command:

```shell
python debug.py -p COM4 -cl yes no up down left right on off
```

(For differnt models the used keywords might have to be adjusted. The COM/serial port will change from system to system)

*Warning:* To enable the debugging output, the MicroKWS project options have to be adjusted. Checkout `micro-kws/debug/README.md` for details!

**Screenshot:**

![screen1.PNG](https://github.com/tum-ei-eda/micro-kws/raw/workshop/resources/debugger/screen1.PNG)